In [1]:
import pandas as pd
import geopandas as gpd
from shapely.wkb import loads as wkb_loads
from shapely.geometry import Polygon
from shapely import wkt
from geowrangler.validation import GeometryValidation
import rasterio
from rasterio.features import shapes

import os
from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Generating Landcover features

The Landcover dataset we will be using is from [ESA WorldCover 2021](https://worldcover2021.esa.int/downloader). Extract the rasters for the lacuna cities before beginning this notebook.

## Set input parameters

In [2]:
RASTERS_PATH = Path("../../../data/02-raw/esa/")

PROJ_CRS = "EPSG:4326"
LOCAL_CRS = "EPSG:3123"

In [3]:
# load raster filenames
esa_rasters = os.listdir(RASTERS_PATH)
print(esa_rasters)

['esa-mandaluyong.tif', 'esa-davao.tif', 'esa-navotas.tif', 'esa-zamboanga.tif', 'esa-muntinlupa.tif', 'esa-palayan.tif', 'esa-dagupan.tif', 'esa-mandaue.tif', 'esa-iloilo.tif', 'esa-cdo.tif', 'esa-legazpi.tif', 'esa-tacloban.tif']


## Utils

In [4]:
def convert_tif_to_polygon(raster_file):
    # open the raster file
    with rasterio.open(raster_file) as src:
        # read the raster data as a numpy array
        raster_data = src.read(1)
        # get the metadata for the raster
        raster_meta = src.meta
    # print(raster_meta)

    # # convert the raster data into polygons
    polygons = []
    for shape, value in shapes(raster_data, transform=raster_meta["transform"]):
        polygons.append({"geometry": shape, "label": value})

    # # convert the polygons to a geopandas dataframe
    gdf = gpd.GeoDataFrame(polygons)

    # # drop NaN values
    gdf = gdf.dropna(subset=["label"])

    # # extract coordinates from the 'geometry' column and convert them into Shapely geometries
    gdf["geometry"] = gdf["geometry"].apply(lambda x: Polygon(x["coordinates"][0]))

    # # reorganize columns
    gdf = gdf[["label", "geometry"]]

    return gdf

## Convert rasters to polygons

In [5]:
%%time

gdf_list = []
for tif_file in esa_rasters:
    gdf = convert_tif_to_polygon(RASTERS_PATH / tif_file)
    gdf = gdf.set_crs(PROJ_CRS)
    gdf_list.append(gdf)

CPU times: user 36 s, sys: 826 ms, total: 36.8 s
Wall time: 37.5 s


In [6]:
gdf_list[3]

,label,geometry
0,30.0,"POLYGON ((122.10433 7.51375, 122.10433 7.51358..."
1,60.0,"POLYGON ((122.10442 7.51375, 122.10442 7.51367..."
2,10.0,"POLYGON ((122.10450 7.51375, 122.10450 7.51367..."
3,60.0,"POLYGON ((122.10475 7.51375, 122.10475 7.51367..."
4,60.0,"POLYGON ((122.10692 7.51375, 122.10692 7.51367..."
...,...,...
51131,95.0,"POLYGON ((122.06183 6.85850, 122.06183 6.85842..."
51132,30.0,"POLYGON ((122.06108 6.85842, 122.06108 6.85833..."
51133,30.0,"POLYGON ((122.06175 6.85850, 122.06183 6.85850..."
51134,30.0,"POLYGON ((122.06208 6.85842, 122.06208 6.85833..."


In [7]:
# concatenate to one dataframe
gdf_big = pd.concat(gdf_list, ignore_index=True)
gdf_big.shape

(333122, 2)

In [8]:
gdf_big["label"].unique()

array([10., 60., 80., 40., 30., 50.,  0., 20., 90., 95.])

In [9]:
# dictionary of the labels definition https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200
# to map to column later
label_map = {
    10: "tree_cover",
    20: "shrubland",
    30: "grassland",
    40: "cropland",
    50: "builtup",
    60: "bare_sparse_vegetation",
    70: "snow_ice",
    80: "permanent_water_bodies",
    90: "herbaceous_wetland",
    95: "mangroves",
}

## Load AOI

In [10]:
aoi = gpd.read_file("../../../data/01-admin-bounds/target_admin_bounds.shp")
aoi.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [11]:
# get barangay area
aoi = aoi.to_crs(LOCAL_CRS)
aoi["brgy_area"] = aoi["geometry"].area
aoi = aoi.to_crs(PROJ_CRS)
aoi = aoi[["ADM4_PCODE", "geometry", "brgy_area"]]

## Computing landcover areas

In [12]:
%%time

aoi_copy = aoi.copy()
landcover = gdf_big.copy()

overlay = aoi_copy.overlay(landcover, how="intersection")
overlay["geometry"] = overlay["geometry"].to_crs(LOCAL_CRS)
overlay["area"] = overlay["geometry"].area

CPU times: user 26.5 s, sys: 265 ms, total: 26.8 s
Wall time: 26.9 s


In [13]:
# sorting of values, dropping duplicates and null values
overlay = overlay.sort_values(by="area", ascending=True)
overlay = overlay.drop_duplicates(subset=["ADM4_PCODE", "label", "area"], keep="last")
overlay = overlay.dropna(subset=["ADM4_PCODE"])
overlay = overlay.sort_values(by=["ADM4_PCODE", "label"], ascending=True)
overlay_merge = overlay.drop("geometry", axis=1)

In [14]:
# groupby label + aggregate total area
# avg_area=('area', 'mean') --- add if needed
label_area = (
    overlay_merge.groupby(["ADM4_PCODE", "label"])
    .agg(area=("area", "sum"))
    .reset_index()
)
label_area = label_area[label_area["label"] != 0]
label_area

,ADM4_PCODE,label,area
0,PH015518001,10.0,2.582837e+05
1,PH015518001,30.0,1.286791e+04
2,PH015518001,40.0,2.471744e+04
3,PH015518001,50.0,1.320202e+05
4,PH015518001,60.0,2.276612e+03
...,...,...,...
4404,PH137603009,30.0,5.857510e+05
4405,PH137603009,40.0,7.905049e+04
4406,PH137603009,50.0,6.925334e+06
4407,PH137603009,60.0,3.183145e+03


In [15]:
# drop overlay geometry from overlay gdf
result = pd.merge(left=aoi, right=label_area, on="ADM4_PCODE", how="left")

In [16]:
result = result.replace({"label": label_map})
result.head(3)

,ADM4_PCODE,geometry,brgy_area,label,area
0,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1.020434e+06,tree_cover,6414.244420
1,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1.020434e+06,grassland,2467.019287
2,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1.020434e+06,cropland,82.233398


In [17]:
result["label"].unique()

array(['tree_cover', 'grassland', 'cropland', 'builtup',
       'bare_sparse_vegetation', 'permanent_water_bodies',
       'herbaceous_wetland', 'shrubland', 'mangroves'], dtype=object)

In [18]:
pivot_labels = result.pivot(index="ADM4_PCODE", columns="label", values="area")
pivot_labels.head(3)

label,bare_sparse_vegetation,builtup,cropland,grassland,herbaceous_wetland,mangroves,permanent_water_bodies,shrubland,tree_cover
ADM4_PCODE,,,,,,,,,
PH015518001,2276.611726,132020.247855,24717.436194,12867.912920,11560.331402,NaN,275370.758662,NaN,258283.688939
PH015518002,780.068685,172224.969082,31337.232598,60137.005807,740.203180,NaN,73612.534745,NaN,269877.786386
PH015518003,283.353301,100768.990730,NaN,82.237679,NaN,NaN,7343.511966,NaN,6461.113968


In [19]:
# join again to main table
result = pd.merge(left=aoi, right=pivot_labels, on="ADM4_PCODE", how="left")

In [20]:
lc_cols = [
    "bare_sparse_vegetation",
    "builtup",
    "cropland",
    "grassland",
    "herbaceous_wetland",
    "mangroves",
    "permanent_water_bodies",
    "shrubland",
    "tree_cover",
]

# get pct cover
for col in lc_cols:
    result[f"pct_{col}"] = 100 * (result[col] / result["brgy_area"])

In [21]:
result = result.drop(columns=["geometry", "brgy_area"])
result.head(3)

,ADM4_PCODE,bare_sparse_vegetation,builtup,cropland,grassland,herbaceous_wetland,mangroves,permanent_water_bodies,shrubland,tree_cover,pct_bare_sparse_vegetation,pct_builtup,pct_cropland,pct_grassland,pct_herbaceous_wetland,pct_mangroves,pct_permanent_water_bodies,pct_shrubland,pct_tree_cover
0,PH015518016,2878.182908,2.343664e+04,82.233398,2467.019287,822.344841,NaN,1.020434e+06,NaN,6414.244420,0.282055,2.296732,0.008059,0.241762,0.080588,NaN,100.000000,NaN,0.628580
1,PH015518031,29473.515344,5.011105e+05,68382.251348,111395.559882,252.081710,NaN,3.199497e+05,NaN,110603.133272,2.826489,48.056129,6.557808,10.682753,0.024174,NaN,30.682941,NaN,10.606759
2,PH015518022,39457.771162,1.019299e+06,53534.393553,126104.095843,2549.277892,NaN,1.959975e+06,82.235824,219657.958010,1.210851,31.279523,1.642825,3.869791,0.078230,NaN,60.146285,0.002524,6.740704


In [22]:
# save file
result_df = pd.DataFrame(result)
result.to_csv("../../../data/04-output/landcover_features_ESA_2021.csv", index=False)